In [5]:
import numpy as np
import pandas as pd

In [6]:
def eliminar_fila_columna(data, i):
    n = data.shape[1]
    ind = np.arange(n)
    ind_new = ind[ind!=i]
    
    W = X[ind_new, :][:,ind_new]
    w = X[i,ind_new]
    w2 = X[i,i]
    return W, w, w2


def soft_threshold_operator(x, t, lim_sup=0):
    return np.sign(x)*np.max([lim_sup,np.abs(x)-t])


In [318]:
def coordinate_descent(X, Y, B_t, Gamma_t, lambda_2, epsilon=1e-5):
    n, p = X.shape
    Beta_ridge = np.abs(np.linalg.inv(X.T@X+lambda_2*np.diag(np.repeat(1,X.shape[1])))@(X.T@Y))
    
    S = X.T@X
    H = X.T@Y@Gamma_t
    
    B_t_new = np.copy(B_t)
    for r in range(B_t.shape[0]):
        for c in range(B_t.shape[1]):
            mu = 0
            for j in range(B_t.shape[0]):
                for k in range(B_t.shape[1]):
                    mu += B_t[j,k]*S[r,j]*Gamma_t[k,c]
            #print(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]))
            B_t_new[r,c] = soft_threshold_operator(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]), 
                                                   n*lambda_2/(S[r,r]*Gamma_t[c,c]))
    #print("rid_dento",np.sum(Beta_ridge))
    while np.sum(np.abs(B_t_new-B_t))>epsilon*np.sum(Beta_ridge):
        #print("dentro", np.sum(np.abs(B_t_new-B_t)))
        B_t = B_t_new
        for r in range(B_t.shape[0]):
            for c in range(B_t.shape[1]):
                mu = 0
                for j in range(B_t.shape[0]):
                    for k in range(B_t.shape[1]):
                        #print(r,c,j,k)
                        mu += B_t[j,k]*S[r,j]*Gamma_t[k,c]
                B_t_new[r,c] = B_t_new[r,c] = soft_threshold_operator(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]), 
                                                   n*lambda_2/(S[r,r]*Gamma_t[c,c]))
    return B_t_new

In [303]:
i = 1
while i >5:
    print(i)
    i+=1

In [325]:
mrce(X, Y, .002, 0.00001, 2)

1376.0605055535482
ridg, [[2.00762497e-03 1.18359118e-03 3.52537807e-04 1.34638706e-05
  1.16548089e-04]
 [9.24229689e+01 5.10028342e+01 6.47755267e+01 5.90976952e+01
  3.67095095e+01]
 [2.46003529e-03 8.51026411e-04 3.49039452e-03 1.07470466e-03
  5.56295288e-04]
 [2.64190024e+01 8.29699758e+01 2.89592459e+00 7.18544026e+01
  5.38742581e+01]
 [3.28015437e-03 6.57395158e-03 4.52184519e-03 6.30510321e-03
  6.31499065e-03]
 [6.29348803e+01 6.41552745e+01 3.11513518e+00 8.05079356e+01
  1.04114352e+01]
 [6.94612576e-04 2.78534806e-03 1.76282113e-04 3.87377873e-03
  8.03043422e-04]
 [1.27653339e+01 4.83527261e+00 9.79939938e-01 5.39372082e+01
  1.81560234e+01]
 [4.75789627e-03 5.16538789e-03 5.51743671e-03 1.78456498e-04
  2.15341719e-03]
 [3.43752193e-03 5.47911054e-05 1.25763245e-03 2.24006914e-04
  1.83161233e-03]
 [3.10769848e-03 6.48988345e-03 3.95610877e-03 4.33802869e-03
  2.96808245e-03]
 [7.18719883e+01 9.41419009e+01 8.75001187e+01 2.81923471e+01
  2.51711722e+01]
 [6.90939298e+0

array([[ 22.95121996, -10.39591337,   7.58892684,   6.51845485,
          3.13510637],
       [ 26.6133136 , -19.58089042,  32.13845952,  50.99805454,
        -14.36644456],
       [-11.20401434,   0.06383404, -14.49104617,  10.13731196,
          5.28385887],
       [-16.24762415,  10.04660936, -17.48848977,   1.0793872 ,
         -3.47665984],
       [-18.95252146,   2.14474262,  -3.79217505,   8.71832897,
          9.74397615],
       [ 34.31076095,   8.79618384, -29.13711603,   7.33686825,
         -8.42162216],
       [ 18.72660029,  -5.83105712,   3.22284541,  -7.17940533,
          5.9325067 ],
       [ -3.81541092, -15.53004128,   8.12568107,  43.63307776,
          5.52853875],
       [ -1.5159772 ,  15.19868111, -14.07647528, -33.68278706,
         -3.79605288],
       [ 24.1986564 ,  -9.0065611 ,  16.35134595,  10.87275695,
        -11.6557053 ],
       [-21.28827816,  11.73784618,   1.41764915,   4.38225834,
        -10.51845677],
       [ -8.05875705,  15.99985002,  39.170

In [309]:
def mrce(X, Y, lambda_1, lambda_2, epsilon):
    # initialize B and Gamma (matrix cov)
    B_t = np.zeros([X.shape[1], Y.shape[1]])
    Gamma_t = graphic_lasso(X, B_t, Y, lambda_1)
    Beta_ridge = np.abs(np.linalg.inv(X.T@X+lambda_2*np.diag(np.repeat(1,X.shape[1])))@(X.T@Y))
    B_t_new = coordinate_descent(X, Y, B_t, Gamma_t, lambda_2, epsilon)
    n = X.shape[1]
    k=0
    print(np.sum(np.abs(B_t_new-B_t)))
    print("ridg,", Beta_ridge)
    while np.sum(np.abs(B_t_new-B_t))>epsilon*np.sum(Beta_ridge):
        k+=1
        print(k)
        B_t_new = coordinate_descent(X, Y, B_t, Gamma_t, lambda_2)
        Gamma_t = graphic_lasso(X, B_t_new, Y, lambda_1)
    return B_t_new

In [198]:
from sklearn.datasets import make_regression
X,Y=make_regression(n_samples=100,n_features=20, n_targets=5,effective_rank=2,random_state=10)
#X_estandarizado=estandarizar.fit_transform(X)
#X=X_estandarizado
B_t = np.zeros([X.shape[1], Y.shape[1]])

In [199]:
Y.shape

(100, 5)

In [237]:
X.shape

(100, 20)

In [238]:
B_t.shape

(20, 5)

In [308]:
def graphic_lasso(X, B, Y, lambda_2, tol_sigma=1e-10, tol_beta=1e-10, max_iter=100):
    n, p = (Y-X@B).shape
    S = (Y-X@B).T@(Y-X@B)/n
    #S_datos = X.T@X
    Sigma = np.linalg.inv(S)
    Sigma_pre = np.zeros(Sigma.shape)
    Sigma_inv = Sigma
    count = 0
    while np.linalg.norm(Sigma - Sigma_pre) <= tol:
        Sigma_pre = Sigma
        for i in range(p):
            W, w, w2  = eliminar_fila_columna(Sigma_inv, i)
            _, s, s_ii = eliminar_fila_columna(S, i)
            _, l, lamb = eliminar_fila_columna(Sigma, i)
            
            beta = np.linalg.inv(W)@w
            beta_prev = np.zeros((p-1,1))
            while np.linalg.norm(beta - beta_prev) <= tol_beta:
                beta_prev = np.copy(beta)
                for j in range(p-1):
                    n = W.shape[1]
                    ind = np.arange(n)
                    ind_new = ind[ind!=j]

                    W_k = W[ind_new, :][:,ind_new]
                    beta_k = beta[ind_new]
                    beta[j] = soft_threshold_operator(s_ii, np.sum(W_k@beta_k), lambda_2)/W[j,j]
                    if i==3:
                        print(beta)
            print(i)
            ind = np.arange(p)
            ind_new = ind[ind!=i]
            w = W@beta
            w2 = s_ii+lambda_2
            Sigma_inv[i,ind_new] = w.T
            Sigma_inv[ind_new, i] = w.T
            Sigma_inv[i,i] = w2
            

            theta22 = 1.0 /(w2-w.T@beta)
            theta12 = -beta*theta22
            # print(theta12)
            
            Sigma[i, ind_new] = theta12.T
            Sigma[ind_new, i] = theta12.T
            Sigma[i,i] = theta22
            
        count += 1
        if count>= max_iter:
            print("Maximo iteraciones")
            break
    return Sigma
    